In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

In [ ]:
import minsearch

In [ ]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

In [ ]:
import json

In [ ]:
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

In [ ]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [ ]:
documents[0]

In [ ]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course'],
)

In [ ]:
# SELECT * WHERE course = 'data-engineering-zoomcamp';

In [ ]:
q = 'the course has already started, can I still enroll?'

In [ ]:
index.fit(documents)

In [ ]:
boost = {
    'question': 3.0,
    #'text': 2,
    'section': 0.5
}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [ ]:
results

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI() # Assumes OPENAI_API_KEY is set in the environment

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": q
        }
    ]
)

In [ ]:
response.choices[0].message.content

In [ ]:
# Now with context
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT to answer. 
If the CONTEXT does not contain the information to answer the QUESTION, say "I don't know".

QUESTION: {question}

CONTEXT: {context}
""".strip()


In [ ]:
context = ""

for doc in results:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"    

In [ ]:
print(context)

In [ ]:
prompt = prompt_template.format(question=q, context=context).strip()
print(prompt)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

In [ ]:
#q = 'the course has already started, can I still enroll?'
response.choices[0].message.content

In [ ]:
def search(query):
    boost = {
        'question': 3.0,
        #'text': 2,
        'section': 0.5
    }

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    
    return results


In [ ]:
def build_prompt(query, search_results):

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT to answer. 

    QUESTION: {question}

    CONTEXT: {context}
    """.strip()

    context = ""

    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"    

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [ ]:
def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return response.choices[0].message.content

In [ ]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
#query = 'the course already already started, can I enroll?'
query = 'how do I run Kafka?'
rag(query)

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es_client = Elasticsearch('http://localhost:9200') # Assumes ES is running locally

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm

In [ ]:

for document in tqdm(documents):
    es_client.index(index=index_name, document=document)

In [ ]:
query = "I just discovered the course, can I still join?"
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [ ]:
response = es_client.search(index=index_name, body=search_query)

In [ ]:
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
elastic_search(query)

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
query = 'how do I run Kafka?'
rag(query)